In [6]:
import csv
import itertools
import pandas as pd
import numpy as np
import networkx as nx

In [7]:
papers=[]
f = open("train.txt", "r")
for x in f:
        token_list=x.rstrip("\n").split()
        papers.append(token_list)


graph=[]

for paper in papers:
    for pair in itertools.combinations(paper, 2):
        graph.append(pair)

graph_df=pd.DataFrame(graph, columns=['Author_A','Author_B'])

#graph_df['AB']=(graph_df['Author_A'],graph_df['Author_B'])


        
graph_df.to_csv('no_headers_graph_pos.csv',header=False,index=False)
g=nx.read_edgelist('no_headers_graph_pos.csv',delimiter=',',create_using=nx.DiGraph(),nodetype=int)
print(nx.info(g))




Name: 
Type: DiGraph
Number of nodes: 3767
Number of edges: 16036
Average in degree:   4.2570
Average out degree:   4.2570


In [8]:
#getting all set of edges
r = csv.reader(open('no_headers_graph_pos.csv','r'))
edges = dict()
for edge in r:
    edges[(edge[0], edge[1])] = 1

In [12]:
%%time
###generating negative edges from given graph
import random
missing_edges = set([])
while (len(missing_edges)<16036): # such that same number as positive edges
    a=random.randint(1, 1862220)
    b=random.randint(1, 1862220)
    tmp = edges.get((a,b),-1)
    if tmp == -1 and a!=b:
        try:
            if nx.shortest_path_length(g,source=a,target=b) > 2: 
                
                missing_edges.add((a,b))
            else:
                continue  
        except:  
                missing_edges.add((a,b))              
    else:
        continue

Wall time: 61.1 ms


In [13]:
len(missing_edges)

16036

In [ ]:
G=nx.MultiGraph()
#G.add_edges_from(graph)

sp = dict(nx.all_pairs_shortest_path(G))
sp

In [15]:
nx.degree_centrality(G)

{'0': 0.016728624535315983,
 '356': 0.018321826872012743,
 '1236': 0.020711630377057883,
 '1655': 0.019383961763143918,
 '1797': 0.012745618693574084,
 '2568': 0.020977164099840677,
 '3649': 0.019118428040361124,
 '3760': 0.013011152416356876,
 '2414': 0.0071694105151354215,
 '1': 0.004248539564524694,
 '19': 0.015400955921402016,
 '141': 0.004514073287307488,
 '337': 0.0055762081784386614,
 '340': 0.004248539564524694,
 '750': 0.004779607010090281,
 '1242': 0.029474243228890067,
 '1309': 0.004514073287307488,
 '1917': 0.016197557089750396,
 '2214': 0.005310674455655868,
 '3393': 0.024694636218799786,
 '3417': 0.004248539564524694,
 '3552': 0.005310674455655868,
 '3630': 0.021508231545406264,
 '3675': 0.010886882634094529,
 '3828': 0.004248539564524694,
 '3951': 0.02602230483271375,
 '2': 0.016994158258098777,
 '51': 0.005045140732873075,
 '64': 0.011949017525225702,
 '1146': 0.004514073287307488,
 '1389': 0.019915029208709505,
 '2585': 0.009028146574614976,
 '2603': 0.0037174721189591

In [18]:
nx.edge_betweenness_centrality(G)

{('0', '356'): 1.6485444873559915e-05,
 ('0', '1236'): 0.00020138218697046063,
 ('0', '1655'): 6.067351183661284e-05,
 ('0', '1797'): 1.6447536142638297e-07,
 ('0', '2568'): 6.390372268045015e-05,
 ('0', '3649'): 0.00012009342045771357,
 ('0', '3760'): 1.6508941353763682e-05,
 ('0', '2414'): 4.159866774187022e-06,
 ('356', '1236'): 4.104685320857721e-05,
 ('356', '1655'): 4.416669872929597e-05,
 ('356', '1797'): 1.6508941353763682e-05,
 ('356', '2568'): 4.198375002932382e-05,
 ('356', '3649'): 4.77682365151235e-05,
 ('356', '3760'): 1.6447536142638297e-07,
 ('356', '3645'): 0.0004278421892502811,
 ('356', '2414'): 4.736276166736965e-05,
 ('1236', '1655'): 0.0001389087566830814,
 ('1236', '1797'): 0.00020695252122363345,
 ('1236', '2568'): 9.635558504821621e-05,
 ('1236', '3649'): 6.328162538544535e-05,
 ('1236', '3760'): 4.661718746174921e-05,
 ('1236', '3645'): 0.00010278126149866981,
 ('1236', '2221'): 8.86347043764473e-06,
 ('1236', '2873'): 0.0005040437651521479,
 ('1236', '3720'):

In [19]:
nx.closeness_centrality(G)

{'0': 0.18787930261293775,
 '356': 0.20191402347287565,
 '1236': 0.22313357423619168,
 '1655': 0.2096826441412535,
 '1797': 0.1878909946474522,
 '2568': 0.20876921539136423,
 '3649': 0.21682013594182475,
 '3760': 0.201927527621048,
 '2414': 0.18135634268319975,
 '1': 0.22944147678318333,
 '19': 0.2550663506792185,
 '141': 0.22944147678318333,
 '337': 0.23148205113776812,
 '340': 0.22944147678318333,
 '750': 0.22944147678318333,
 '1242': 0.27343057353648886,
 '1309': 0.2420798903936746,
 '1917': 0.24472889624624378,
 '2214': 0.2394306417914282,
 '3393': 0.25717379838074184,
 '3417': 0.22944147678318333,
 '3552': 0.23390303633327467,
 '3630': 0.2659637414543613,
 '3675': 0.2365048091015126,
 '3828': 0.22944147678318333,
 '3951': 0.2830696036930348,
 '2': 0.26318169394960855,
 '51': 0.2396777322370334,
 '64': 0.2458248162343193,
 '1146': 0.2386546828701217,
 '1389': 0.2836280312813443,
 '2585': 0.24548503073338557,
 '2603': 0.2384661869512605,
 '585': 0.24709226556918812,
 '1872': 0.25640

In [1]:
# usage of link prediction specific library from networkX.
nx.algorithms.link_prediction.adamic_adar_index(G)

NameError: name 'nx' is not defined